<a href="https://colab.research.google.com/github/ludwigwittgenstein2/Research/blob/master/Hugging_Face_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hugging Face Tutorial 

1. Sentimenent Analysis
2. Sequence Classification
3. Extractive Question Answering
4. Open Ended text generation
4. Named Entity Recognition

In [2]:
!pip install transformers
# https://huggingface.co/transformers/task_summary.html



     |████████████████████████████████| 1.8MB 11.5MB/s 
     |████████████████████████████████| 890kB 51.2MB/s 
     |████████████████████████████████| 3.2MB 44.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=09029f92808aabed05b15bcc0b5ce94ef0c6cf5370a68360d7fe7ea288692f33
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


1.Sentiment Analysis 

We use this to find positive or negative sentiment in reviews, comments about customers 

2. Sequence Classification

We use 


#Sentiment Analysis

In [3]:
#Import Modules
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [ ]:

classifier('We are very sad to show you the 😜 Transfomers Library')


[{'label': 'NEGATIVE', 'score': 0.9984051585197449}]

In [ ]:
results = classifier(["We are very happy to show you the 🤫 Transfomers library", "we hope you don't hate it"])

for result in results: 
  print(f"label:{result['label']}, with score:{round(result['score'], 4)}")

label:POSITIVE, with score:0.9998
label:POSITIVE, with score:0.832


In [ ]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")


In [ ]:
#Sequence Classification

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
classifier = pipeline('sentiment-analysis', model = "nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
classifier

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)




In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name= "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
#Tokenize them into numbers, next build a tensor out of them
inputs = tokenizer("We are happy to show you the Transformers library.")
print(inputs)



{'input_ids': [101, 2057, 2024, 3407, 2000, 2265, 2017, 1996, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
pt_batch = tokenizer(["We are happy to show you the Transfomers Library.", "We hope you don't hate it."],
                     padding=True, truncation=True, return_tensors="pt")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
for key, value in pt_batch.items():
  print(f"{key}:{value.numpy().tolist()}")

input_ids:[[101, 2057, 2024, 3407, 2000, 2265, 2017, 1996, 9099, 14876, 16862, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask:[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


In [ ]:
#Now the input has been preprocessed by tokenizer
pt_outputs = pt_model(**pt_batch)

In [ ]:
print(pt_outputs)

(tensor([[-4.1925,  4.4667],
        [ 0.0818, -0.0418]], grad_fn=<AddmmBackward>),)


In [ ]:
#Let's apply SoftMax to this
import torch.nn.functional as F
pt_predictions = F.softmax(pt_outputs[0], dim=-1)
print(pt_predictions)


tensor([[1.7351e-04, 9.9983e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward>)


In [ ]:
#tokenizer.save_pretrained(save_directory)
#model.save_pretrained(save_directory)
#We can load back this 
#tokenizer = AutoTokenizer.from_pretrained(save_directory)
#model = TFAutoModel.from_pretrained(save_directory, from_pt=True)

#Sequence Classification

We classify sequences according to given number of classes 



In [ ]:
#Let's try Sequence Classification
from transformers import pipeline 
nlp = pipeline("sentiment-analysis")
result = nlp("I hate you")[0]
print(f"label:{result['label']},with score:{round(result['score'],4)}")

label:NEGATIVE,with score:0.9991


In [ ]:
#Sequence Classificaiton 
#1.Instantiate Tokenizer and Model from Checkpoint. Model is BERT
#2. We Build a Sequence 
#3. Pass Sequence through the Model 
#4. Compute Softmax of Result
#5. Print Results 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")


Should be paraphrase
not paraphrase: 10%
is paraphrase: 90%

Should not be paraphrase
not paraphrase: 94%
is paraphrase: 6%


#Extractive Question Answering

In [ ]:
#Extractive Question Answering
from transformers import pipeline 

nlp = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset, 
which is entirely based on that task.
"""
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: 'the task of extracting an answer from a text given a question.', score: 0.59, start: 34, end: 96


In [ ]:
#Another Example # Extractive Question Answering

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
... architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
... Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
... TensorFlow 2.0 and PyTorch.
"""

questions = ["How many pretrained models are available in Transformers?", 
             "What does Transformers provide?", 
             "Transformers provide interoperability between which frameworks?"]
             

In [ ]:
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)
    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in Transformers?
Answer: over 32 +
Question: What does Transformers provide?
Answer: general - purpose . . . architectures
Question: Transformers provide interoperability between which frameworks?
Answer: tensorflow 2 . 0 and pytorch


In [ ]:
#Masked Language Modeling

from transformers import pipeline 
nlp = pipeline("fill-mask")


Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from pprint import pprint 
pprint(nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks."))


[{'score': 0.17927460372447968,
  'sequence': '<s>HuggingFace is creating a tool that the community uses to '
              'solve NLP tasks.</s>',
  'token': 3944,
  'token_str': 'Ġtool'},
 {'score': 0.1134939044713974,
  'sequence': '<s>HuggingFace is creating a framework that the community uses '
              'to solve NLP tasks.</s>',
  'token': 7208,
  'token_str': 'Ġframework'},
 {'score': 0.05243545398116112,
  'sequence': '<s>HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.</s>',
  'token': 5560,
  'token_str': 'Ġlibrary'},
 {'score': 0.03493543714284897,
  'sequence': '<s>HuggingFace is creating a database that the community uses '
              'to solve NLP tasks.</s>',
  'token': 8503,
  'token_str': 'Ġdatabase'},
 {'score': 0.02860247902572155,
  'sequence': '<s>HuggingFace is creating a prototype that the community uses '
              'to solve NLP tasks.</s>',
  'token': 17715,
  'token_str': 'Ġprototype'}]


In [ ]:
#Attribute Error! 

from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelWithLMHead.from_pretrained("distilbert-base-cased")

sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."
input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits

mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

In [ ]:
#Causal Language Modelling 
#Attribute Error, Torch Update

from transformers import AutoModelWithLMHead, AutoTokenizer, top_k_top_p_filtering
import torch 
from torch.nn import functional as F 

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")

sequence = f"Trump or Biden will win the Election, Who will win"
input_ids = tokenizer.encode(sequence, return_tensors="pt")

next_token_logits = model(input_ids).logits[:,-1,:]
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

probs = F.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([])
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

#sample
probs = F.softmax(siltered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)
generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])


# Text Generation
Open Ended Text Generation
  Goal is to create coherent portion of text in given    context

Text Generation is possible with GPT-2, OpenAi-GPT, CTRL, PyTorch, Tensorflow


In [ ]:
from transformers import pipeline
text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("xlnet-base-cased")

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")


# Padding text helps XLNet with short prompts 

PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""


prompt = "Today the weather is really nice and I am planning on "

inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))


outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)

generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

In [ ]:
print(generated)

Today the weather is really nice and I am planning on anning on posting a daily blog entry in the middle of the day as it will take less than a couple of hours to make it back to the hotel to clean up the room for the day. I have a few extra pictures this week too, but I don't have it to go to the bathroom that is the most important thing. I am hoping the next day has a


# Named Entity Recognition

O, Outside of a named entity

B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity

I-MIS, Miscellaneous entity

B-PER, Beginning of a person’s name right after another person’s name

I-PER, Person’s name

B-ORG, Beginning of an organisation right after another organisation

I-ORG, Organisation

B-LOC, Beginning of a location right after another location

I-LOC, Location

In [ ]:
from transformers import pipeline
nlp = pipeline("ner")

sequence = "Who will win the Election? Biden or Trump? We are excited to see"


In [ ]:
print(nlp(sequence))

[{'word': 'Election', 'score': 0.9869940280914307, 'entity': 'I-MISC', 'index': 5}, {'word': 'B', 'score': 0.9516230225563049, 'entity': 'I-PER', 'index': 7}, {'word': '##iden', 'score': 0.9853422045707703, 'entity': 'I-PER', 'index': 8}, {'word': 'Trump', 'score': 0.9972804188728333, 'entity': 'I-PER', 'index': 10}]


In [ ]:
from transformers import pipeline 

nlp = pipeline("ner")

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very" "close to the Manhattan Bridge which is visible from the window."


In [ ]:
print(nlp(sequence))

[{'word': 'Hu', 'score': 0.9995632767677307, 'entity': 'I-ORG', 'index': 1}, {'word': '##gging', 'score': 0.9915938377380371, 'entity': 'I-ORG', 'index': 2}, {'word': 'Face', 'score': 0.9982671737670898, 'entity': 'I-ORG', 'index': 3}, {'word': 'Inc', 'score': 0.9994403719902039, 'entity': 'I-ORG', 'index': 4}, {'word': 'New', 'score': 0.9994346499443054, 'entity': 'I-LOC', 'index': 11}, {'word': 'York', 'score': 0.9993270635604858, 'entity': 'I-LOC', 'index': 12}, {'word': 'City', 'score': 0.9993864893913269, 'entity': 'I-LOC', 'index': 13}, {'word': 'D', 'score': 0.9825621843338013, 'entity': 'I-LOC', 'index': 19}, {'word': '##UM', 'score': 0.936983048915863, 'entity': 'I-LOC', 'index': 20}, {'word': '##BO', 'score': 0.8987101316452026, 'entity': 'I-LOC', 'index': 21}, {'word': 'Manhattan', 'score': 0.9758241176605225, 'entity': 'I-LOC', 'index': 29}, {'word': 'Bridge', 'score': 0.9902493953704834, 'entity': 'I-LOC', 'index': 30}]


In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
"O",       # Outside of a named entity
"B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
"I-MISC",  # Miscellaneous entity
"B-PER",   # Beginning of a person's name right after another person's name
"I-PER",   # Person's name
"B-ORG",   # Beginning of an organisation right after another organisation
"I-ORG",   # Organisation
"B-LOC",   # Beginning of a location right after another location
"I-LOC"    # Location
]
sequence = "Hugging Face is a company based in New York City. It's HQ is in Manhattan Bridge"
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")
outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)


In [ ]:

#print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])

#Summarizer

Summarizing the text into shorter text 



In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """
New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
 A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
 Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
 In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
 Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
 2010 marriage license application, according to court documents.
 Prosecutors said the marriages were part of an immigration scam.
 On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
 After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
 Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
 All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
 Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
 Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
 The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
 Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
 Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
 If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
 """



In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))


[{'summary_text': ' Liana Barrientos has been married 10 times, nine of them between 1999 and 2002 . Prosecutors say the marriages were part of an immigration scam . She is believed to still be married to four men and at one time, she was married to eight men at once .'}]


#Translation

From English to German

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")
print(translator("Who will win the Election? Biden or Trump?", max_length=40))



Some weights of T5Model were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'translation_text': 'Wer wird die Wahl gewinnen: Biden oder Trump?'}]
